In [1]:
import os, re
import numpy as np
import tensorflow as tf

file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()
    
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [3]:
def change_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 변경, 양쪽 공백 삭제
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽 공백 삽입
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백 하나로
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 문자를 하나의 공백으로 변경
    sentence = sentence.strip() # 양쪽 공백 삭제
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에 <start>, 끝에 <end>를 추가한다.
    return sentence
    
print(change_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if sentence[-1] == ":":
        continue
    
    changed_sentence = change_sentence(sentence)
    corpus.append(changed_sentence)
    
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=7000, filters='', oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer
    
tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f5fea13bf40>


In [6]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10:
        break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [8]:
source_input = tensor[:, :-1]
target_input = tensor[:, 1:]

print(source_input[0])
print(target_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [9]:
BUFFER_SIZE = len(source_input)
BATCH_SIZE = 256
steps_per_epoch = len(source_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((source_input, target_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
        
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [13]:
for source_sample, target_sample in dataset.take(1):
    break

model(source_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-4.64363227e-04,  1.61830278e-04, -3.14673467e-04, ...,
          4.21569595e-04,  2.52897298e-04, -1.32707180e-04],
        [-8.26138305e-04, -9.70383044e-05, -3.48264526e-04, ...,
          5.59910201e-04,  1.45858095e-04,  1.31319694e-05],
        [-1.29641965e-03, -2.81053624e-04, -2.56651314e-04, ...,
          7.75502063e-04,  2.01173767e-04,  2.69232027e-04],
        ...,
        [-1.81228854e-03, -2.30299099e-03,  3.20459087e-03, ...,
          9.67934902e-04, -1.30347034e-03,  3.76898068e-04],
        [-2.26381375e-03, -2.25120666e-03,  3.66648799e-03, ...,
          9.80452402e-04, -1.47522951e-03,  3.46892659e-04],
        [-2.68977555e-03, -2.16462673e-03,  4.08664485e-03, ...,
          9.81136574e-04, -1.60059496e-03,  3.17072991e-04]],

       [[-4.64363227e-04,  1.61830278e-04, -3.14673467e-04, ...,
          4.21569595e-04,  2.52897298e-04, -1.32707180e-04],
        [-5.26450109e-04, -2.72942358e-04, -5.

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 38s 384ms/step - loss: 3.6467
Epoch 2/30
93/93 [==============================] - 36s 385ms/step - loss: 2.8487
Epoch 3/30
93/93 [==============================] - 36s 386ms/step - loss: 2.7719
Epoch 4/30
93/93 [==============================] - 36s 386ms/step - loss: 2.7011
Epoch 5/30
93/93 [==============================] - 36s 385ms/step - loss: 2.6312
Epoch 6/30
93/93 [==============================] - 36s 387ms/step - loss: 2.5744
Epoch 7/30
93/93 [==============================] - 36s 387ms/step - loss: 2.5285
Epoch 8/30
93/93 [==============================] - 36s 387ms/step - loss: 2.4869
Epoch 9/30
93/93 [==============================] - 25s 268ms/step - loss: 2.4491
Epoch 10/30
93/93 [==============================] - 17s 181ms/step - loss: 2.4110
Epoch 11/30
93/93 [==============================] - 17s 181ms/step - loss: 2.3760
Epoch 12/30
93/93 [==============================] - 17s 180ms/step - loss: 2.3407
Epoch 13/30
9

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token:
            break
        if test_tensor.shape[1] >= max_len: 
            break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [26]:
# generate_text(model, tokenizer, init_sentence="<start> he")
generate_text(model, tokenizer, init_sentence="<start> I")
# generate_text(model, tokenizer, init_sentence="<start> she")
# generate_text(model, tokenizer, init_sentence="<start>")

'<start> i ll tell you , sir , i ll tell you . <end> '